In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import json
import time
import homebrew as hb
%load_ext autoreload
%autoreload 2

In [2]:
with open('/Users/patrickfuller/.secrets/spotify_api.json') as f:
    creds = json.load(f)
client_id = creds['client_id']
key = creds['api_key']

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                      client_secret=key)

spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
cat_list = []
response = spotify.categories(country='US', limit=50)
items = response['categories']['items']
for item in items:
    category = item['id']
    cat_list.append(category)

In [10]:
for cat in ['family', 'comedy', 'word', 'ellen', 'sleep']:   
    cat_list.remove(cat)

In [12]:
master_playlist_dict = {}
for category in cat_list:
    offset = 0
    while True:
        response = spotify.category_playlists(category_id=category,
                                                             offset=offset,
                                                             limit=50, country='US',
                                                             )
        current_names_ids = hb.extract_name_and_id(response)
        if current_names_ids == {}:                     # If we run out of playlists in a category
            break                                       # we move on to the next category
        master_playlist_dict.update(current_names_ids)
        offset += 50

In [25]:
with open('playlists_artists_redundant.json', 'w') as f:
    for pl_id, pl_data in list(master_playlist_dict.items()):
        pl_artists = []
        offset = 0
        for _ in range(2):
            try:
                response = spotify.user_playlist_tracks(user=pl_data['owner_id'],
                                             playlist_id=pl_id,
                                             limit=100)
                current_artists = hb.get_artists_in_playlist(response)
                pl_artists.extend(current_artists)
                unique = set(pl_artists)
                for artist in unique:
                    line = {
                        'artist':(artist),
                        'playlist':(pl_data['name'], pl_id),
                    }
                    json.dump(line, f)
                    f.write('\n')
            except:
                pass